In [1]:
import netCDF4 as nc
import glob
import numpy as np
fs=glob.glob("output/subset_*.nc")
fs=sorted(fs)
y_lastbin=[]

x_data=[]
y_data=[]
p_type_data=[]
n_seq_data=[]
#print(fs)
y_nonz=[[]for k in range(44+18)]
x_nonz=[[]for k in range(44+18)]
xL=[]
yL=[]
diffL=[]
for f in fs:
    with nc.Dataset(f) as fh:
        bin_nodes=fh.variables['bin_nodes'][:]
        diffL.extend(bin_nodes[:,-1]-(bin_nodes[:,2]-2))

print(np.array(diffL).min(),np.array(diffL).max())


-22 22


In [2]:
import matplotlib.pyplot as plt
#h=plt.hist(diffL,bins=20)
print(len(fs))

357


In [18]:
n_files=len(fs)
x_data=[]
y_data=[]
x_pia_data=[]
y_pia_data=[]
p_type_data=[]
n_seq_data=[]
bin_data=[]
z_meas2L=[]
bin_nodesL=[]
pia_dataL=[]
import tqdm
for f in tqdm.tqdm(fs[:n_files//2]):
    with nc.Dataset(f) as fh:
        
        vars=fh.variables.keys()
        z_meas=fh.variables['z_meas'][:]
        p_type=fh.variables['p_type'][:]
        bin_nodes=fh.variables['bin_nodes'][:]
        sfc_bin=fh.variables['sfc_bin'][:]
        p_rate_cmb=fh.variables['p_rate_cmb'][:]
        dm=fh.variables['dm'][:]
        bin_zero_deg=fh.variables['bin_zero_deg'][:]
        bin_storm_top=fh.variables['bin_storm_top'][:]
        surface_type=fh.variables['surface_type'][:]
        pia=fh.variables['pia'][:]
        z_meas2=np.log10(0.5*10**(z_meas[:,::2]/10)+0.5*10**(z_meas[:,1::2]/10)+1e-9)*10
        z_meas2[z_meas2<0]=0
        for i,z_meas_1d in enumerate(z_meas2):
            if surface_type[i]!=0:
                continue
            if p_type[i]==3:
                continue
            itop=bin_storm_top[i]//2
            x_1d=[]
            y_1d=[]
            i_valid=0
            bin_1d=[]
            for k1 in range(bin_nodes[i,0],bin_nodes[i,-1]): 
                x_1d.append([z_meas_1d[k1],k1-bin_nodes[i,2]])
                y_1d.append([np.log10(1+p_rate_cmb[i,k1]/0.1),dm[i,k1]])
                bin_1d.append(k1-bin_nodes[i,2])
                i_valid+=1
            
            if(i_valid>0):
                n_seq_data.append(i_valid)
                bin_data.extend(bin_1d)
                x_data.append(x_1d)
                k1=bin_nodes[i,-1]
                x_pia_data.append([sfc_bin[i,0]-bin_nodes[i,-1]])
                y_pia_data.append([pia[i,0],np.log10(1+p_rate_cmb[i,k1]/0.1),dm[i,k1]])
                y_data.append(y_1d)
                p_type_data.append(p_type[i])
                z_meas2L.append(z_meas_1d.copy())
                bin_nodesL.append(bin_nodes[i,:])
    #break

#vars='z_meas', 'p_type', 'bin_nodes', 'sfc_bin', 'p_rate_cmb', 'dm', 'bin_zero_deg', 'bin_storm_top'


100%|██████████| 178/178 [01:24<00:00,  2.10it/s]


In [4]:
print(len(x_data))
print(np.max(n_seq_data))

663803
68


In [19]:
#plt.plot(x_data[25000][:,0])
n_all=len(x_data)
print(n_all)
x_data_unpacked=np.zeros((n_all,72,2),np.float32)
y_data_unpacked=np.zeros((n_all,72,2),np.float32)
z_meas2L=np.array(z_meas2L)
bin_nodesL=np.array(bin_nodesL)
x_pia_data=np.array(x_pia_data)
y_pia_data=np.array(y_pia_data)
for i in range(n_all):
    nz=len(x_data[i])
    x_data_unpacked[i,:nz,:]=np.array(x_data[i])
    x_data_unpacked[i,:nz,:1]=(x_data_unpacked[i,:nz,:1]-12)/8
    x_data_unpacked[i,:nz,1:2]/=8.0
    y_data_unpacked[i,:nz,:]=np.array(y_data[i])

print(x_pia_data.shape) 


663803
(663803, 1)


In [6]:
import numpy as np
ix=np.random.randint(0,n_all)
#ix=3000
print(z_meas2L.shape,n_all)

for k in range(bin_nodesL[ix,0],bin_nodesL[ix,-1]):
    x1=[z_meas2L[ix][k],k-bin_nodesL[ix,2]]
    x1[0]=(x1[0]-12)/8
    x1[1]=x1[1]/8
    xL.append(x1)
    print(np.array(x1),x_data_unpacked[ix,k-bin_nodesL[ix,0],:])

(663803, 88) 663803
[ 0.17286491 -0.875     ] [ 0.17286491 -0.875     ]
[ 0.37375569 -0.75      ] [ 0.3737557 -0.75     ]
[ 0.06635046 -0.625     ] [ 0.06635046 -0.625     ]


In [9]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class LSTMPackedModel(nn.Module):
    def __init__(self, input_size, second_input_size, hidden_size, num_layers, output_size, second_output_size, n_comp):
        super(LSTMPackedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Final output layer
        self.sec_out_fc1=nn.Linear(second_input_size+hidden_size,hidden_size)
        self.sec_out_fc2=nn.Linear(hidden_size,(n_comp+2)*second_output_size)
    def forward(self, x, lengths, x2):
        # Pack the padded sequence
        packed_input = rnn_utils.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        
        # LSTM forward pass
        packed_output, (hn, cn) = self.lstm(packed_input)
        print(packed_output.data.shape)

        # Unpack the sequence
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)
        print(output.shape)
        n_batch = len(lengths)
        batch_indices = torch.arange(n_batch)
        last_output = output[batch_indices,lengths-1,:]  
        last_output=torch.cat([last_output,x2],dim=1)
        # Apply the output layer to the LSTM output
        output_1 = self.fc(output)
        hid1=self.sec_out_fc1(last_output)
        # apply relu
        hid1=torch.relu(hid1)
        output_2 = self.sec_out_fc2(hid1)

        return output_1, output_2


In [20]:

# define an ensemble of LSTMPackedModel models 


# Example usage:
# Assume input is a padded batch with variable sequence lengths
batch_size = 4
input_size = 2
second_input_size = 1
second_output_size = 3
hidden_size = 20
num_layers = 2
output_size = 2
n_comp=2
seq_lengths = torch.tensor([41, 21, 12, 33]) 
padded_input = torch.randn(batch_size, seq_lengths.max(), input_size)
second_input = torch.randn(batch_size,second_input_size) # Variable lengths of the sequences
input_size = 2


model = LSTMPackedModel(input_size, second_input_size, hidden_size, num_layers, output_size, second_output_size, n_comp)

padded_input = torch.tensor(x_data_unpacked[:10,:,:])
seq_lengths = torch.tensor(n_seq_data[:10])
second_input = torch.tensor(x_pia_data[:10])


# Forward pass
output_1, output_2 = model(padded_input, seq_lengths, second_input)

print(output_1.shape)
print(output_2.shape)


torch.Size([21, 20])
torch.Size([10, 3, 20])
torch.Size([10, 3, 2])
torch.Size([10, 12])


In [22]:
print(output_1.shape)
print(output_2.shape)
print(y_pia_data[:10].shape)

torch.Size([10, 3, 2])
torch.Size([10, 12])
(10, 3)


In [25]:
from torch.distributions.lowrank_multivariate_normal import LowRankMultivariateNormal
def NLLLoss(output, target, sec_out_size):
    nb, n2 = output.shape
    n1=sec_out_size
    ncomp=n2//sec_out_size-2
    mean = output[:,0:n1]
    cov_diag = torch.exp(output[:,n1:2*n1])+0.01
    factors = output[:,2*n1:].reshape(nb, n1, ncomp)
    dist = LowRankMultivariateNormal(mean, factors, cov_diag)
    return -dist.log_prob(target).mean()

target = torch.tensor(y_pia_data[:10],dtype=torch.float32)
loss = NLLLoss(output_2, target, second_output_size)

In [26]:
x_data_n=np.array(x_data_unpacked)
y_data_n=np.array(y_data_unpacked)
print(x_data_n.shape)
print(y_data_n.shape)
x_data_n=x_data_n
y_data_n=y_data_n
n_ens=5
n_all=len(x_data_n)
n_all_sub=n_all//n_ens

x_data_t=torch.tensor(x_data_n,dtype=torch.float32)
y_data_t=torch.tensor(y_data_n,dtype=torch.float32)
x_pia_data_t=torch.tensor(x_pia_data,dtype=torch.float32)
y_pia_data_t=torch.tensor(y_pia_data,dtype=torch.float32)
print(x_data_t.shape)
n_seq_data_t=torch.tensor(n_seq_data,dtype=torch.int32)
p_type_data=np.array(p_type_data)
a=np.nonzero(p_type_data>0)[0]


(663803, 72, 2)
(663803, 72, 2)
torch.Size([663803, 72, 2])


In [30]:
print(p_type_data[a].shape)
a=np.nonzero(p_type_data==2)[0]
#print(n_seq_data)
x_data_strat_n=torch.tensor(x_data_n[a][::2],dtype=torch.float32)
y_data_strat_n=torch.tensor(y_data_n[a][::2],dtype=torch.float32)
x_pia_data_strat=torch.tensor(np.array(x_pia_data)[a][::2],dtype=torch.float32)
y_pia_data_strat=torch.tensor(np.array(y_pia_data)[a][::2],dtype=torch.float32)
n_seq_data_strat=torch.tensor(np.array(n_seq_data)[a][::2],dtype=torch.int32)

strat_dataL=[]
strat_loaderL=[]
n_all_str=x_data_strat_n.shape[0]
print(n_all_str)
n_all_sub=n_all_str//n_ens


(136953,)
68477


In [ ]:

for i in tqdm.tqdm(range(n_ens)):
    strat_data=torch.utils.data.TensorDataset(x_data_strat_n[i*n_all_sub:(i+1)*n_all_sub,:,:],y_data_strat_n[i*n_all_sub:(i+1)*n_all_sub,:,:],n_seq_data_strat[i*n_all_sub:(i+1)*n_all_sub],x_pia_data_strat[i*n_all_sub:(i+1)*n_all_sub,:],y_pia_data_strat[i*n_all_sub:(i+1)*n_all_sub,:])
    print(i,strat_data.tensors[0].shape)
    strat_loader=torch.utils.data.DataLoader(strat_data,batch_size=64,shuffle=True)
    strat_dataL.append(strat_data)
    strat_loaderL.append(strat_loader)
    #print(i,strat_data.tensors[0].shape)


100%|██████████| 5/5 [00:00<00:00, 2246.31it/s]

0 torch.Size([13695, 72, 2])
0 torch.Size([13695, 72, 2])
1 torch.Size([13695, 72, 2])
1 torch.Size([13695, 72, 2])
2 torch.Size([13695, 72, 2])
2 torch.Size([13695, 72, 2])
3 torch.Size([13695, 72, 2])
3 torch.Size([13695, 72, 2])
4 torch.Size([13695, 72, 2])
4 torch.Size([13695, 72, 2])


In [31]:
n_epoch=15
model_list=[]
#
for i in tqdm.tqdm(range(n_ens)):
    model_list.append(LSTMPackedModel(2,64,3,2))
    model_strat=model_list[i]#
    strat_criterion=nn.MSELoss(reduction='none')
    strat_optimizer=torch.optim.Adam(model_strat.parameters(),lr=0.001)

    for epoch in range(n_epoch):
        running_loss=0
        for x,y,n_seq in strat_loaderL[i]:
            strat_optimizer.zero_grad()
            y_pred=model_strat(x,n_seq)
            n_seq1=y_pred.shape[1]
       
            loss=strat_criterion(y_pred,y[:,:n_seq1,:])
            mask=torch.arange(n_seq1).expand(y.shape[0],n_seq1)<n_seq.unsqueeze(1)
        
            masked_loss=(loss*mask.unsqueeze(2)).sum()
            masked_loss=masked_loss/mask.sum()
    
            masked_loss.backward()
            running_loss+=masked_loss.item()
            strat_optimizer.step()
        print(epoch,running_loss/len(strat_loader))
        

  0%|          | 0/5 [00:00<?, ?it/s]

0 0.06934201862895861
1 0.02420530130621046
2 0.022658148509799504
3 0.021992281836719484
4 0.021407379979791585
5 0.02106362636623089
6 0.020827438218111637
7 0.02064113713131519
8 0.020419841608963908
9 0.020350829797098412
10 0.020209105785033897
11 0.020079684624943184
12 0.020033100412547356
13 0.019952269347413676


 20%|██        | 1/5 [15:31<1:02:04, 931.01s/it]

14 0.019891637776163408
0 0.06796584181574872
1 0.02512463169041439
2 0.02286907410962158
3 0.022096279269317166
4 0.021575996471074176
5 0.021125539940112504
6 0.020791261498379753
7 0.020570674612827133
8 0.020421929088479375
9 0.020307379881705855
10 0.02021378282806836
11 0.020082360033120493
12 0.020005288698303047
13 0.019943869152484694


 40%|████      | 2/5 [31:19<47:03, 941.29s/it]  

...

14 0.021358884300570934
0 0.06954631688713561
1 0.025609556872950634
2 0.024016199359903113
3 0.02324647347049904
4 0.02259410867554834
5 0.02222356697457144
6 0.021936603450012627
7 0.021678154290566454
8 0.021577572362002684
9 0.02137623948874534
10 0.021355465260421626
11 0.021240044703881723
12 0.02113234385906253
13 0.021025945502333342


 80%|████████  | 4/5 [1:01:42<15:19, 919.40s/it]

14 0.020987325462920127
0 0.06751989782205783
1 0.027066640510020078
2 0.025383431928639767
3 0.024577124524512328
4 0.024035189689311665
5 0.02353460068407003
6 0.023254785277822522
7 0.022990844793821453
8 0.022773481126205298
9 0.02265720828145277
10 0.02255013150133891
11 0.022430843003530754
12 0.022356770417536607
13 0.02227863672451349


100%|██████████| 5/5 [1:17:24<00:00, 928.94s/it]

14 0.02221139175380813


In [65]:

for epoch in range(n_epoch2):
    running_loss=0
    for x,y,n_seq in strat_loader:
        strat_ku_optimizer.zero_grad()
        y_pred=model_ku_strat(x[:,:,a_feat],n_seq)
        n_seq1=y_pred.shape[1]
       
        loss=strat_criterion(y_pred,y[:,:n_seq1,:2])
        mask=torch.arange(n_seq1).expand(y.shape[0],n_seq1)<n_seq.unsqueeze(1)
        
        masked_loss=(loss*mask.unsqueeze(2)).sum()
        masked_loss=masked_loss/mask.sum()
    
        masked_loss.backward()
        running_loss+=masked_loss.item()
        strat_ku_optimizer.step()
    print(epoch,running_loss/len(strat_loader))

0 0.03449178049541849
1 0.03447961658284032
2 0.03437462921881919
3 0.034343278710004525
4 0.03427321770467418
5 0.03410242947836896
6 0.03394827105069189
7 0.03378847528615301
8 0.033944862818778704
9 0.03381727487563515


In [32]:
for i in range(n_ens):
    torch.onnx.export(model_list[i],(x[:1][:,:,:],n_seq[:1]),'strat_ku_ocean_feb4_2025_%2.2i.onnx'%i,export_params=True,opset_version=11,do_constant_folding=True,input_names=['input','n_seq'],output_names=['output'])

[W214 09:49:07.968910000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W214 09:49:07.979180000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W214 09:49:07.189027000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PadPacked type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W214 09:49:08.349283000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it i

In [55]:
model_strat.eval()
output=model_strat(x_data_strat_n[::2,:,:],n_seq_data_strat[::2])

In [58]:
output_np=output.detach().numpy()
import pickle
pickle_out={"output":output_np,"y_data":y_data_strat_n[::2,:,:].detach().numpy(),"x_data":x_data_strat_n[::2,:,:].detach().numpy(),"n_seq":n_seq_data_strat[::2].detach().numpy(),"z_meas":z_meas2_strat[::2],"bin_nodes":bin_nodesL_strat[::2]}
pickle.dump(pickle_out,open("output_strat_jan28_2025.pkl","wb"))

In [70]:
torch.onnx.export(model_ku_strat,(x[:1][:,:,a_feat],n_seq[:1]),'conv_and_strat_model_ku_jan28_2025.onnx',export_params=True,opset_version=11,do_constant_folding=True,input_names=['input','n_seq'],output_names=['output'])

[W129 11:23:13.674287000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W129 11:23:13.675937000 shape_type_inference.cpp:1999] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
[W129 11:23:13.885182000 shape_type_inference.cpp:1999] Warning: The shape 